### 필요한 개발환경 및 패키지 설치
- conda create -n st python=3.8
- pip install sentence-transformers
- conda install -n st ipykernel --update-deps --force-reinstall
- conda install pandas
- conda install ipywidgets

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

### SentenceBERT 모델 로드

In [2]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [3]:
sentences = ['안녕하세요?', '한국어 문장 임베딩을 위한 버트 모델입니다.']
embeddings = model.encode(sentences)
print(embeddings)

[[-0.37510458 -0.7733844   0.5927712  ...  0.5792354   0.32683465
  -0.6508966 ]
 [-0.09361678 -0.18191552 -0.1923082  ... -0.03165798  0.30412537
  -0.26793647]]


In [4]:
embeddings.shape

(2, 768)

### 데이터셋 로드
- 웰니스 대화 스크립트 데이터셋


In [5]:
df = pd.read_csv('wellness_dataset_original.csv')
df.head()

,구분,유저,챗봇,Unnamed: 3
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,NaN
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,NaN
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",NaN
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,NaN


In [6]:
df.shape

(5231, 4)

- 전처리

In [7]:
df = df.drop(columns=['Unnamed: 3'])
df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN


In [8]:
df = df.dropna()
df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.


In [9]:
df.shape

(1034, 3)

- 인코딩

In [10]:
model.encode(df['유저'][0])

array([-4.80605900e-01, -2.94869393e-01,  4.37900096e-01, -6.40137792e-01,
        3.28670628e-02, -3.42647493e-01, -5.47480583e-02,  1.73052475e-02,
       -4.08220917e-01, -5.06034255e-01, -1.68733329e-01, -3.98677140e-01,
       -1.24776825e-01, -9.71540660e-02, -1.65286273e-01,  5.72626200e-03,
        6.13689683e-02, -1.91312313e-01,  2.53917158e-01, -5.85019648e-01,
       -2.84425855e-01, -2.32035190e-01, -3.27080786e-01,  6.72973171e-02,
       -1.65573183e-05, -4.72336203e-01, -3.60022068e-01,  2.91879892e-01,
       -6.63861394e-01, -3.10574591e-01,  5.79525173e-01, -3.11722696e-01,
        1.47700924e-02, -2.12172821e-01,  2.22057775e-01, -1.73829064e-01,
       -3.78458530e-01, -4.20398176e-01, -2.38218799e-01,  6.38705641e-02,
       -1.15304112e-01, -2.44563922e-01, -5.00228465e-01,  1.68355435e-01,
       -6.58360064e-01, -8.91942084e-01, -6.26957059e-01, -3.21965277e-01,
       -7.05359042e-01,  3.71447295e-01, -5.45803249e-01,  7.76302293e-02,
        1.09864689e-01,  

In [11]:
df['embedding'] = pd.Series([[]] * len(df))     # dummy
df.shape

(1034, 4)

In [12]:
df['embedding'] = df['유저'].map(lambda x: list(model.encode(x)))
df.head(3)

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806059, -0.2948694, 0.4379001, -0.6401378..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561571, -0.14506254, 0.2949032, -0.673949..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652, -0.081268236, 1.0945568, 0.10579172,..."


In [13]:
df.to_csv('wellness_dataset.csv', index=False)

### 간단한 챗봇

In [14]:
text = '요즘 머리가 아프고 너무 힘들어'
embedding = model.encode(text)

In [15]:
# squeeze()는 리스트 형태의 값을 스칼라 값으로 변환해줌
df['유사도'] = df.embedding.map(lambda x: cosine_similarity([embedding], [x]).squeeze())
df.head()

,구분,유저,챗봇,embedding,유사도
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806059, -0.2948694, 0.4379001, -0.6401378...",0.448967
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561571, -0.14506254, 0.2949032, -0.673949...",0.490199
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652, -0.081268236, 1.0945568, 0.10579172,...",0.352131
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.7679057, 0.46520707, 0.52850693, -0.507604...",0.422284
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277727, -0.37413934, 0.040532183, -0.862...",0.315118


In [16]:
answer = df.loc[df.유사도.idxmax()]
answer

구분                                                      증상/편두통
유저                            요즘은 머리가 한쪽만 지그시 누르는 것처럼 무겁고 아파요.
챗봇                           으으, 머리가 아프면 정말 힘들죠. 그 마음 정말 이해해요.
embedding    [-0.6085822, 0.06865987, 0.43937874, -0.217735...
유사도                                                   0.829629
Name: 4871, dtype: object

In [17]:
# 답변할 내용
answer.챗봇

'으으, 머리가 아프면 정말 힘들죠. 그 마음 정말 이해해요.'

### 저장한 파일 확인

In [18]:
import json
df = pd.read_csv('wellness_dataset.csv')
df.head(3)

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806059, -0.2948694, 0.4379001, -0.6401378..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561571, -0.14506254, 0.2949032, -0.673949..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652, -0.081268236, 1.0945568, 0.10579172,..."


In [19]:
df['embedding'] = df.embedding.apply(json.loads)    # 문자열을 숫자 데이터로 변환
df.head(3)

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806059, -0.2948694, 0.4379001, -0.6401378..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561571, -0.14506254, 0.2949032, -0.673949..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652, -0.081268236, 1.0945568, 0.10579172,..."
